# **DCI CHALENGE**

Here I am going to create a database in SQLite, using python to acchieve this. 

First let's create all the required tables in python.

In [21]:
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta

### **Terminal table**

In [22]:
data = {
    'ID': range(1, 11),
    'terminal_name': [
        'Terminal 1', 
        'Terminal 2', 
        'Terminal 3', 
        'Terminal 4', 
        'Terminal 5', 
        'Terminal 6', 
        'Terminal 7', 
        'Terminal 8', 
        'Terminal 9', 
        'Terminal 10'
    ]
}

terminal = pd.DataFrame(data)

terminal

,ID,terminal_name
0,1,Terminal 1
1,2,Terminal 2
2,3,Terminal 3
3,4,Terminal 4
4,5,Terminal 5
5,6,Terminal 6
6,7,Terminal 7
7,8,Terminal 8
8,9,Terminal 9
9,10,Terminal 10


### **Opening hours**

In [23]:
data = []
for i in range(10):  # for each terminal
    opening_hour_weekday = random.choice(60 * np.array([7, 8, 9]))  # opening between 7:00 AM and 9:00 AM
    closing_hour_weekday = random.choice(60 * np.array([17, 18, 19]))  # closing between 5:00 PM and 7:00 PM
    opening_hour_weekend = opening_hour_weekday # same as weekday
    closing_hour_weekend = random.choice(60 * np.array([12, 13, 14]))  # closing earlier between 12:00 PM and 2:00 PM
    for day in range(7):  # For each day of the week
        if day < 5:
            data.append({
                'terminal_id': terminal['ID'][i],
                'day_of_week': day,
                'opening_hour': opening_hour_weekday,
                'closing_hour': closing_hour_weekday
            })
        else:
            data.append({
                'terminal_id': terminal['ID'][i],
                'day_of_week': day,
                'opening_hour': opening_hour_weekend,
                'closing_hour': closing_hour_weekend
            })

opening_hours = pd.DataFrame(data)

opening_hours.head(21)

,terminal_id,day_of_week,opening_hour,closing_hour
0,1,0,480,1020
1,1,1,480,1020
2,1,2,480,1020
3,1,3,480,1020
4,1,4,480,1020
5,1,5,480,720
6,1,6,480,720
7,2,0,480,1080
8,2,1,480,1080
9,2,2,480,1080


### **Orders**

Let's simulate 10k orders

In [24]:
order_ids = range(1, 10001) # integer from 1 to 10.000

terminal_ids = np.random.randint(1, 11, size=10000)

# generate random timestamps within a specific date range. In this case, in 2023
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 1, 1)

def random_date(start, end):
    return start + timedelta(
        seconds=np.random.randint(0, int((end - start).total_seconds()))
    )

timestamps = [random_date(start_date, end_date) for _ in range(10000)]

In [25]:
orders = pd.DataFrame({
    'id': order_ids,
    'terminal_id': terminal_ids,
    'timestamp': timestamps
})

orders

,id,terminal_id,timestamp
0,1,8,2023-08-12 21:25:24
1,2,1,2023-02-02 19:29:42
2,3,8,2023-01-29 04:07:12
3,4,7,2023-07-24 20:05:54
4,5,6,2023-02-14 18:25:10
...,...,...,...
9995,9996,1,2023-06-15 06:31:16
9996,9997,10,2023-02-19 23:47:32
9997,9998,3,2023-11-08 11:56:34
9998,9999,7,2023-05-18 18:23:28


### **SQL to the rescue**

The next step is to tell python to create a SQL database so we can persist our fresh new data and run our queries

In [26]:
import sqlite3

In [27]:
with sqlite3.connect('DCI.db') as conn:
    cursor = conn.cursor()

    cursor.execute('DROP TABLE IF EXISTS terminal')
    cursor.execute('''
        CREATE TABLE terminal (
            ID INTEGER PRIMARY KEY,
            terminal_name TEXT NOT NULL
        )
    ''')

    # insert data into the 'terminal' table
    terminal.to_sql('terminal', conn, if_exists='replace', index=False)

    cursor.execute('DROP TABLE IF EXISTS opening_hours')
    cursor.execute('''
        CREATE TABLE opening_hours (
            terminal_id INTEGER,
            day_of_week INTEGER,
            opening_hour INTEGER,
            closing_hour INTEGER,
            FOREIGN KEY(terminal_id) REFERENCES terminal(ID)
        )
    ''')

    # Insert data into the 'opening_hours' table
    opening_hours.to_sql('opening_hours', conn, if_exists='replace', index=False)

    cursor.execute('DROP TABLE IF EXISTS orders')
    cursor.execute('''
        CREATE TABLE orders (
            id INTEGER PRIMARY KEY,
            terminal_id INTEGER,
            timestamp DATETIME,
            FOREIGN KEY(terminal_id) REFERENCES terminal(ID)
        )
    ''')

    # Insert data into the 'orders' table
    orders.to_sql('orders', conn, if_exists='replace', index=False, dtype={'timestamp': 'DATETIME'})

Finally, let's run our final query to get the answer

In [28]:
with sqlite3.connect('DCI.db') as conn:
    cursor = conn.cursor()

    cursor.execute('''
        WITH orders_formatted AS (
            SELECT 
                terminal_id,
                STRFTIME('%w', timestamp) AS day_of_week,
                (STRFTIME('%H', timestamp) * 60 + STRFTIME('%M', timestamp)) AS order_time
            FROM orders
        )
        , orders_within_opening_hours AS (
            SELECT 
                of.terminal_id,
                t.terminal_name,
                of.day_of_week,
                oh.opening_hour,
                oh.closing_hour,
                of.order_time,
                CASE 
                    WHEN of.order_time < oh.opening_hour THEN 1
                    ELSE 0
                END AS before_opening_hour,
                CASE 
                    WHEN of.order_time BETWEEN oh.opening_hour AND oh.closing_hour THEN 1
                    ELSE 0
                END AS within_hours,
                CASE 
                    WHEN of.order_time > oh.closing_hour THEN 1
                    ELSE 0
                END AS after_closing_hour
            FROM orders_formatted of
            JOIN opening_hours oh
            ON of.terminal_id = oh.terminal_id AND of.day_of_week = oh.day_of_week
            JOIN terminal t
            ON t.id = oh.terminal_id
        )

        SELECT 
            terminal_id,
            terminal_name,
            SUM(before_opening_hour) AS before_opening_hour_orders,
            SUM(within_hours) AS within_hours_orders,
            SUM(after_closing_hour) AS after_closing_hour_orders
        FROM orders_within_opening_hours
        GROUP BY terminal_id
    ''')

    rows = cursor.fetchall()
    col_names = [desc[0] for desc in cursor.description]
    
    result = pd.DataFrame(rows, columns=col_names)

result

,terminal_id,terminal_name,before_opening_hour_orders,within_hours_orders,after_closing_hour_orders
0,1,Terminal 1,314,320,358
1,2,Terminal 2,319,341,317
2,3,Terminal 3,325,362,324
3,4,Terminal 4,271,423,271
4,5,Terminal 5,277,378,338
5,6,Terminal 6,291,412,273
6,7,Terminal 7,336,346,376
7,8,Terminal 8,398,278,335
8,9,Terminal 9,264,391,352
9,10,Terminal 10,316,408,286
